#### This notebook takes a directory of 100 mpeg-4 RGB movies whose fps are all equal to 24 and are 2 seconds long, and creates the dataset that was used to train the Boris AI Denoiser model with denoise.ipynb. Each movie is split into 96x96 pixel crops(220 cropped videos per movie) so each sub-directory represents 1 cropped video and is named `$movie_index-$crop_index`. Each sub-directory has 48 numbered frames written as png files.

In [ ]:
import moviepy.editor as mpy
from PIL import Image
import os
import numpy as np

In [ ]:
os.mkdir('datasets/denoise')
directory = os.listdir('datasets/2k_video_clips')
crop_size = 96
movie_index = 0
for video_filename in directory:
    movie = mpy.VideoFileClip(os.path.join('datasets/2k_video_clips', video_filename))
    frame_index = 0
    for frame in movie.iter_frames(fps = 24, dtype='uint8'):
        # make sure there are no more than 48 frames per movie
        if frame_index > 47:
            continue
        image = Image.fromarray(frame)
        width, height = image.size
        num_horizontal_crops = width // crop_size
        num_vertical_crops = height // crop_size
        crop_index = 0
        for i in range(num_horizontal_crops):
            for j in range(num_vertical_crops):
                left, top, right, bottom = i*crop_size, j*crop_size, (i+1)*crop_size, (j+1)*crop_size
                cropped_image = image.crop((left, top, right, bottom))
                filepath = os.path.join('datasets/denoise','%02d-%03d' %(movie_index, crop_index), '%02d' %frame_index + '.png')
                if frame_index == 0:
                    os.mkdir('datasets/denoise/%02d-%03d' %(movie_index, crop_index))
                cropped_image.save(filepath)
                crop_index += 1
        frame_index += 1
    movie_index += 1
                                